In [ ]:
import jax.numpy as jnp
import jax.random as jrand
from jax import make_jaxpr

import genjax
from genjax import gen, pjax

In [ ]:
pjax(genjax.categorical.sample, jrand.key(20))(jnp.array([2.0, 0.1]))

In [ ]:
pjax(genjax.normal.sample, jrand.key(1))(0.0, 1.0)

In [ ]:
@gen
def model():
    x = genjax.normal(0.0, 1.0) @ "x"
    return x

In [ ]:
make_jaxpr(model.simulate)(())

In [ ]:
make_jaxpr(pjax(model.simulate, jrand.key(1)))(())

In [ ]:
model.simulate(())